## Training

In [1]:
import os

In [3]:
%pwd

'c:\\Users\\harik\\Desktop\\All Folder\\PGDA\\Interships & Projects\\NLP\\End-to-End Workflow\\cnnClassifier\\research'

In [31]:
# os.chdir("c:\\Users\\harik\\Desktop\\All Folder\\PGDA\\Interships & Projects\\NLP\\End-to-End Workflow/cnnClassifier")

In [4]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\harik\\Desktop\\All Folder\\PGDA\\Interships & Projects\\NLP\\End-to-End Workflow\\cnnClassifier'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen= True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


from cnnClassifier.entity import PrepareCallbackConfig

# @dataclass(frozen= True)
# class PrepareCallbackConfig:
#     root_dir: Path
#     tensorboard_root_log_dir: Path
#     checkpoint_model_filepath: Path

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml, create_directories
import tensorflow as tf

In [9]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])


    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
          
          config = self.config.prepare_callbacks
          model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)   # Now it ignores the extension (model.h5) and only creating the dir

          create_directories([
                Path(model_ckpt_dir),
                Path(config.tensorboard_root_log_dir)  # Here we dont need to create root dir again as it will be automatically created  with these codes                                                        
          ])

          prepare_callback_config = PrepareCallbackConfig(               # Now we calling the entity we prepared before
                root_dir= Path(config.root_dir),
                tensorboard_root_log_dir= Path(config.tensorboard_root_log_dir),
                checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
          )

          return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
          training= self.config.training
          prepare_base_model= self.config.prepare_base_model
          params= self.params
          training_data= os.path.join(self.config.data_ingestion.unzip_dir, "PetImages")
          
          create_directories([
                Path(training.root_dir)
          ])

          training_config= TrainingConfig(
                root_dir= Path(training.root_dir),
                trained_model_path= Path(training.trained_model_path),
                updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
                training_data= Path(training_data),
                params_epochs= params.EPOCHS,
                params_batch_size= params.BATCH_SIZE,
                params_is_augmentation= params.AUGMENTATION,
                params_image_size= params.IMAGE_SIZE
          )

          return training_config

In [10]:
import time

In [11]:
from cnnClassifier.components import PrepareCallback


# class PrepareCallback:

#     def __init__(self, config: PrepareCallbackConfig):
#         self.config= config


#     @property
#     def _create_tb_callback(self):
#         timestamp= time.strftime("%Y-%m-%d-%H-%M-%S")
#         tb_running_log_dir= os.path.join(
#             self.config.tensorboard_root_log_dir,
#             f"tb_logs_at_{timestamp}"
#         )
#         return tf.keras.callbacks.TensorBoard(log_dir= tb_running_log_dir)
    
#     """ 
#         When we use this property decorator, if we are calling this fuction (object) else where we dont actually need to specify the () to call
#         it. We can access this object like its a attribute.

#     """

#     @property
#     def _create_ckpt_callbacks(self):
#         return tf.keras.callbacks.ModelCheckpoint(
#             filepath= str(self.config.checkpoint_model_filepath),
#             save_best_only= True
#         )
    
#     def get_tb_ckpt_callbacks(self):
#         return[
#             self._create_tb_callback,
#             self._create_ckpt_callbacks
#         ]

In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [13]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config= config

    def get_base_model(self):
        self.model= tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs= dict(
            rescale= 1./255,
            validation_split= 0.20
        )

        dataflow_kwargs= dict(
            target_size= self.config.params_image_size[:-1],
            batch_size= self.config.params_batch_size,
            interpolation= "bilinear"

        )

        valid_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator= valid_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= "validation",
            shuffle= False
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range= 40,
                horizontal_flip= True,
                width_shift_range= 0.2,
                height_shift_range= 0.2,
                shear_range= 0.2,
                zoom_range= 0.2,
                **datagenerator_kwargs
            )

        else:
            train_datagenerator= valid_datagenerator

        self.train_generator= train_datagenerator.flow_from_directory(
            directory= self.config.training_data,
            subset= "training",
            shuffle= True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    def train(self, callback_list: list):
        self.steps_per_epoch= self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps= self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs= self.config.params_epochs,
            steps_per_epoch= self.steps_per_epoch,
            validation_steps= self.validation_steps,
            validation_data= self.valid_generator,
            callbacks= callback_list
        )

        self.save_model(
            path= self.config.trained_model_path,
            model= self.model
        )

In [14]:
try:
    config= ConfigurationManager()
    prepare_callback_config= config.get_prepare_callback_config()
    prepare_callbacks= PrepareCallback(config= prepare_callback_config)
    callback_list= prepare_callbacks.get_tb_ckpt_callbacks()

    training_config= config.get_training_config()
    training= Training(config= training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list= callback_list
    )

except Exception as e:
    raise e

[2023-10-13 08:29:57,876: INFO: common: yaml file: config\config.yaml loaded succesfully]
[2023-10-13 08:29:57,881: INFO: common: yaml file: params.yaml loaded succesfully]
[2023-10-13 08:29:57,881: INFO: common: created directory at: artifacts]
[2023-10-13 08:29:57,881: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-10-13 08:29:57,881: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-10-13 08:29:57,890: INFO: common: created directory at: artifacts\training]


TypeError: unsupported operand type(s) for ** or pow(): 'bool' and 'dict'